In [1]:
!pip install surprise
import pandas as pd
import numpy as np

import surprise
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import warnings; warnings.simplefilter('ignore')

     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675705 sha256=60b129166124393eafd5ac7638ab9cf7dbe30c4966edd80f606e7bd05f87c8d0
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [28]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
rootdir = '/content/drive/My Drive/Colab Notebooks/Movie  Recommendation System/Movie Dataset/'

In [5]:
ratings = pd.read_csv(rootdir+'ratings.csv')

In [6]:
ratings.shape

(100836, 4)

In [7]:
movies = pd.read_csv(rootdir+'movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


**Data Preprocessing**

In [8]:
for dataset in [ratings,movies]:
  dataset.drop_duplicates(inplace=True)

In [9]:
ratings[ratings['userId'].isnull()]

,movieId,title,userId,rating


In [10]:
ratings[ratings['rating'].isnull()]

,movieId,title,userId,rating


**CF based recommendation system**

Our content based engine suffers from some severe limitations.

* It is only capable of suggesting movies which are close to a certain movie. That is, it is not capable of capturing tastes and providing recommendations across genres.
* Also, the engine that we built is not really personal in that it doesn't capture the personal tastes and biases of a user. Anyone querying our engine for recommendations based on a movie will receive the same recommendations for that movie, regardless of who (s)he is.
* Therefore, in this section, we will use Collaborative Filtering to make recommendations to Movie Watchers. Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.


**User based**

In [11]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [13]:
userRatings = userRatings.replace(0.0,np.nan)

In [14]:
userRatings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [22]:
def normalize(row):
    new_row = (row - np.nanmin(row))/(np.nanmax(row)-np.nanmin(row))
    return new_row

* since each user has its own standard we can't assume A's rating 7 is same as another users' 7. So we use Correlation similarity. We use Shifted Cosine similarity also known as Pearson correlation

In [23]:
shiftedratings = userRatings.apply(normalize,axis=1)
shiftedratings = shiftedratings.replace(np.nan,0.0)
shiftedratings

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),21 (2008),21 Grams (2003),21 Jump Street (2012),22 Jump Street (2014),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),30 Days of Night (2007),300 (2007),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",48 Hrs. (1982),50 First Dates (2004),...,"Wolf of Wall Street, The (2013)","Wolverine, The (2013)",Women on the Verge of a Nervous Breakdown (Mujeres al borde de un ataque de nervios) (1988),Wonder Boys (2000),Wonder Woman (2017),Working Girl (1988),"World Is Not Enough, The (1999)",World War Z (2013),"World's End, The (2013)",Wreck-It Ralph (2012),"Wrestler, The (2008)",Wyatt Earp (1994),"X-Files: Fight the Future, The (1998)",X-Men (2000),X-Men Origins: Wolverine (2009),X-Men: Apocalypse (2016),X-Men: Days of Future Past (2014),X-Men: First Class (2011),X-Men: The Last Stand (2006),X2: X-Men United (2003),Yellow Submarine (1968),Yes Man (2008),Yojimbo (1961),You Don't Mess with the Zohan (2008),You Only Live Twice (1967),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.75,0.00,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.000000,0.000,...,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,1.000000,0.0,0.000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.0,0.0,0.5,0.000,0.0,0.0,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.75
2,0.0,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.000000,0.000,...,1.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.200,0.000000,0.00,0.000000,0.000000,0.00
3,0.0,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.000000,0.000,...,0.000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00
4,0.0,0.000,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.0,0.000,0.0,0.000,0.0,0.000000,0.000,...,0.000,0.00,0.000000,0.750000,0.0,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000,0.000000,0.00,0.000000,0.000000,0.00
5,0

In [24]:
cosine_sim = cosine_similarity(shiftedratings,shiftedratings)
cosine_sim

array([[1.        , 0.02836341, 0.10772391, ..., 0.30928693, 0.08985299,
        0.19600048],
       [0.02836341, 1.        , 0.        , ..., 0.04445808, 0.02254747,
        0.12539581],
       [0.10772391, 0.        , 1.        , ..., 0.02880632, 0.        ,
        0.05144036],
       ...,
       [0.30928693, 0.04445808, 0.02880632, ..., 1.        , 0.10593065,
        0.41306226],
       [0.08985299, 0.02254747, 0.        , ..., 0.10593065, 1.        ,
        0.05358715],
       [0.19600048, 0.12539581, 0.05144036, ..., 0.41306226, 0.05358715,
        1.        ]])

In [25]:
resultDf = pd.DataFrame(cosine_sim,index=shiftedratings.index,columns=shiftedratings.index)

In [26]:
resultDf

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.028363,0.107724,0.210737,0.132746,0.149210,0.171259,0.138791,0.081415,0.016575,0.140005,0.006719,0.111444,0.113493,0.172450,0.171151,0.293437,0.244729,0.363360,0.171535,0.172192,0.053179,0.119255,0.166458,0.105936,0.119029,0.245142,0.241803,0.152512,0.104595,0.179607,0.155707,0.174926,0.086470,0.071125,0.079215,0.137909,0.140875,0.336969,0.097251,...,0.086113,0.136553,0.266757,0.097674,0.082289,0.054044,0.337828,0.000000,0.138569,0.269656,0.061421,0.051673,0.055122,0.098284,0.122060,0.132048,0.134003,0.188573,0.118421,0.347129,0.145902,0.151140,0.179961,0.168021,0.121616,0.143619,0.343582,0.012414,0.369345,0.329646,0.090789,0.176934,0.282791,0.073845,0.164514,0.221192,0.282597,0.309287,0.089853,0.196000
2,0.028363,1.000000,0.000000,0.000000,0.006396,0.022968,0.020628,0.012665,0.000000,0.072945,0.039278,0.000000,0.051191,0.006701,0.087672,0.087667,0.088805,0.163520,0.011908,0.014829,0.095419,0.134342,0.007041,0.126960,0.242716,0.000000,0.000000,0.063268,0.096094,0.123735,0.000000,0.008410,0.033584,0.036283,0.000000,0.049542,0.011224,0.011018,0.000000,0.008308,...,0.000000,0.030572,0.134412,0.017301,0.000000,0.000000,0.007228,0.000000,0.000000,0.059267,0.107079,0.193904,0.033197,0.000000,0.138086,0.091052,0.000000,0.042182,0.011887,0.058834,0.038766,0.020178,0.055611,0.015028,0.000000,0.116679,0.015131,0.063079,0.119303,0.023781,0.190184,0.008165,0.015516,0.000000,0.000000,0.035649,0.006178,0.044458,0.022547,0.125396
3,0.107724,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015651,0.053945,0.000000,0.049722,0.029851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052177,0.000000,0.000000,0.000000,0.017078,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021002,0.000000,0.000000,0.000000,0.000000,0.021669,0.063598,0.000000,0.000000,0.056857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021850,0.000000,0.000000,0.037168,0.000000,0.000000,0.000000,0.000000,0.000000,0.049217,0.044382,0.000000,0.077958,0.011924,0.000000,0.000000,0.051894,0.000000,0.020835,0.017728,0.025089,0.028806,0.000000,0.051440
4,0.210737,0.000000,0.000000,1.000000,0.133283,0.093137,0.123735,0.055778,0.013377,0.035055,0.056257,0.037670,0.073319,0.037431,0.074514,0.161520,0.131645,0.137303,0.232513,0.127167,0.053390,0.041100,0.081689,0.076515,0.035757,0.072952,0.137942,0.159126,0.064347,0.062833,0.140917,0.219534,0.187598,0.039434,0.042474,0.129300,0.062694,0.087587,0.256229,0.049975,...,0.024284,0.195731,0.131420,0.069384,0.076051,0.000000,0.190215,0.000000,0.095492,0.146980,0.037644,0.043201,0.045170,0.038406,0.090745,0.063885,0.135933,0.052981,0.062566,0.244036,0.111047,0.069360,0.226238,0.087982,0.076140,0.118798,0.307611,0.017065,0.263594,0.242969,0.084272,0.133324,0.363936,0.054598,0.105810,0.254152,0.138693,0.151353,0.038753,0.142048
5,0.132746,0.006396,0.000000,0.133283,1.000000,0.325061,0.108252,0.418228,0.000000,0.031912,0.181183,0.042420,0.010222,0.192691,0.108217,0.082358,0.177555,0.130699,0.104622,0.107710,0.061245,0.030904,0.056945,0.094824,0.049852,0.271225,0.103645,0.139680,0.061678,0.055594,0.072383,0.238058,0.312907,0.024932,0.296088,0.023083,0.267268,0.363866,0.149699,0.317272,...,0.023899,0.129118,0.126122,0.290206,0.076342,0.000000,0.123399,0.000000,0.187476,0.118675,0.069978,0.000000,0.131753,0.321277,0.036495,0.068355,0.165345,0.361554,0.232315,0.162129,0.000000,0.354574,0.165862,0.153612,0.079616,0.104847,0.117274,0.000000,0.120679,0.143652,0.080109,0.421900,0.132865,0.256905,0.163368,0.139252,0.151270,0.132824,0.303915,0.075010


In [27]:
def get_similar_users(user):
    if user not in resultDf.index:
        return None
    else:
        similarList = zip(resultDf[user],list(resultDf.columns.values))
        similarList = sorted(similarList, key=lambda x: x[0], reverse=True)
        return similarList[1:10]
get_similar_users(609)

[(0.4629100498862757, 340),
 (0.4535573676110726, 565),
 (0.43495883620083997, 54),
 (0.42869105772339466, 512),
 (0.4247953952428732, 242),
 (0.40768712416360564, 379),
 (0.40120542714251967, 8),
 (0.39339789623472154, 133),
 (0.3911149739941278, 37)]